In [42]:
import pandas as pd
import numpy as np
import re

In [46]:
d=pd.read_csv('global_warming_2020.csv',index_col='Unnamed: 0')
d['probas']=d['probas'].apply(lambda x: re.sub('[\[\]\'\s]','',x).split(','))
d['pr']=d['probas'].apply(lambda x: [x[i] for i in range(len(x)) if i%2==1])
d['au_pr']=d['probas'].apply(lambda x: [x[i] for i in range(len(x)) if i%2==0])
d=d[d['pr'].apply(lambda x: all([len(x[i])>1  for i in range(len(x))]))].copy() #deleting empty probas
d=d[d['pr'].apply(lambda x: all([x[i][0]=='0' for i in range(len(x))]))].copy() #deleting wrong parsed probas
d=d[d['pr'].apply(lambda x: float(x[0])-float(x[1])>0.3)] #deleting uncertained authors
d=d[d['speaker']!='None'].copy() #deleting None author
df_out=d[['quotation','speaker','qids','numOccurrences','urls']] #leaving only usefull columns
print('Size of the cleaned dataframe is',df_out.shape)

Size of the cleaned dataframe is (13089, 5)


In [47]:
#merging with information about speakers
dff=pd.DataFrame()
for i in range(0,16):
    if i<10:
        s='0'+str(i)
    else:
        s=str(i)
    df=pd.read_parquet('speaker_attributes/part-000'+s+'-0d587965-3d8f-41ce-9771-5b8c9024dce9-c000.snappy.parquet')
    dff=pd.concat([dff,df_out.merge(df[['gender', 'nationality', 'ethnic_group', 'academic_degree', 'religion','label']],how='left',left_on='speaker',right_on='label').dropna(subset=['gender'])]).copy()
#getting final dataframe
dff=dff.drop_duplicates(subset=['speaker','quotation']).copy()
print('Size of the dataframe with information about authors is',dff.shape)

Size of the dataframe with information about authors is (12167, 11)


In [48]:
#final dataframe
dff.head()

,quotation,speaker,qids,numOccurrences,urls,gender,nationality,ethnic_group,academic_degree,religion,label
3,"This budget also allows us to earmark £ 500,00...",John Whitehead,"['Q55436000', 'Q58150131', 'Q6263827', 'Q62638...",1,['https://www.buryfreepress.co.uk/news/draft-b...,[Q6581097],[Q30],None,None,None,John Whitehead
7,would represent a significant increase in rene...,Tim Smith,"['Q19519946', 'Q29568415', 'Q3528691', 'Q56853...",4,['https://www.sunjournal.com/2020/01/01/trump-...,[Q6581097],[Q16],None,None,None,Tim Smith
12,Make the federal commitment to fund public tra...,Catherine McKenna,['Q21335996'],2,['https://rabble.ca/blogs/bloggers/david-suzuk...,[Q6581072],[Q16],None,None,None,Catherine McKenna
13,That's pretty much the same picture globally. ...,Chris Wright,"['Q18347494', 'Q23303387', 'Q2964899', 'Q30612...",2,['https://www.duluthnewstribune.com/news/gover...,[Q6581097],[Q30],None,None,None,Chris Wright
14,The proposed Thal Sena Bhawan will bring all A...,Manoj Mukund Naravane,['Q44633840'],1,['https://www.thehindu.com/news/national/army-...,[Q6581097],None,None,None,None,Manoj Mukund Naravane
